# LinearRegression from Scratch


# 구현할 것
- 공부시간과 성적간의 관계를 모델링한다.
    - **머신러닝 모델(모형)이란** 수집한 데이터를 기반으로 입력값(Feature)와 출력값(Target)간의 관계를 하나의 공식으로 정의한 함수이다. 그 공식을 찾는 과정을 **모델링**이라고 한다.
    - 이 예제에서는 공부한 시험시간으로 점수를 예측하는 모델을 정의한다.
    - 입력값과 출력값 간의 관계를 정의할 수있는 다양한 함수(공식)이 있다. 여기에서는 딥러닝과 관계가 있는 **Linear Regression** 을 사용해본다.

# 데이터 확인
- 입력데이터: 공부시간
- 출력데이터: 성적

|공부시간|점수|
|-|-|
|1|20|
|2|40|
|3|60|

우리가 수집한 공부시간과 점수 데이터를 바탕으로 둘 간의 관계를 식으로 정의 할 수 있으면 **내가 몇시간 공부하면 점수를 얼마 받을 수 있는지 예측할 수 있게 된다.**   
수집한 데이터를 기반으로 앞으로 예측할 수있는 모형을 만드는 것이 머신러닝 모델링이다.

  

## 학습(훈련) 데이터셋 만들기
- 모델을 학습시키기 위한 데이터셋을 구성한다.
- 입력데이터와 출력데이터을 각각 다른 행렬로 구성한다.
- 하나의 데이터 포인트의 입력/출력 값은 같은 index에 정의한다.

### 선형회귀 (Linear Regression)
- Feature들의 가중합을 이용해 Target을 추정한다.
- Feature에 곱해지는 가중치(weight)들은 각 Feature가 Target 얼마나 영향을 주는지 영향도가 된다.
    - 음수일 경우는 target값을 줄이고 양수일 경우는 target값을 늘린다.
    - 가중치가 0에 가까울 수록 target에 영향을 주지 않는 feature이고 0에서 멀수록 target에 많은 영향을 준다.
- 모델 학습과정에서 가장 적절한 Feature의 가중치를 찾아야 한다.
      

\begin{align}
&\large \hat{y} = W\cdot X + b\\
&\small \hat{y}: \text{모델추정값}\\
&\small W: \text{가중치}\\
&\small X: \text{Feature(입력값)}\\
&\small b: \text{bias(편향)}
\end{align}



## Train dataset 구성
- Train data는 feature(input)와 target(output) 각각 2개의 행렬로 구성한다.
- Feature의 행은 관측치(개별 데이터)를 열을 Feature(특성, 변수)를 표현한다. 이 문제에서는 `공부시간` 1개의 변수를 가진다.
- Target은 모델이 예측할 대상으로 행은 개별 관측치, 열은 각 항목에 대한 정답으로 구성한다.   
  이 문제에서 예측할 항목은 `시험점수` 한개이다.

In [1]:
import numpy as np
study_hours = [[1], [2], [3]]
scores = [[20],[40],[60]]

np.shape(study_hours), np.shape(scores)

((3, 1), (3, 1))

In [13]:
# tensor로 정의
import torch
X_train = torch.tensor(study_hours, dtype=torch.float32)
y_train = torch.tensor(scores, dtype=torch.float32)

In [15]:
print(X_train.shape, y_train.size())
print(X_train.dtype, y_train.dtype)
X_train.type()

torch.Size([3, 1]) torch.Size([3, 1])
torch.float32 torch.float32


'torch.FloatTensor'

## 파라미터 (weight, bias) 정의
- 학습대상/최적화 대상

In [10]:
X_train.size()

torch.Size([3, 1])

In [22]:
# torch.manual_seed(0)
# y_pred = X_train * weight + bias
weight = torch.randn(1, 1, requires_grad=True) #shape: (1:X의 feature개수, 1:출력값(예측)의 개수)
bias = torch.randn(1, requires_grad=True)

print(weight.shape, bias.shape)
print(weight, bias)

torch.Size([1, 1]) torch.Size([1])
tensor([[-0.6871]], requires_grad=True) tensor([-1.5204], requires_grad=True)


In [23]:
# 추론 (w * x + b)
pred = X_train @ weight + bias
pred

tensor([[-2.2075],
        [-2.8946],
        [-3.5817]], grad_fn=<AddBackward0>)

In [13]:
y_train

tensor([[20.],
        [40.],
        [60.]])

In [18]:
pred - y_train

tensor([[-20.8624],
        [-41.9991],
        [-63.1358]], grad_fn=<SubBackward0>)

In [26]:
# 오차 계산 - MSE
loss = torch.mean((y_train - pred) ** 2)
loss

tensor(2125.2480, grad_fn=<MeanBackward0>)

In [27]:
# weight, bias의 gradient계산
loss.backward()

In [28]:
weight.grad, bias.grad

(tensor([[-199.1611]]), tensor([-85.7891]))

In [29]:
# weight 변경(update)
## new_weight = weight - weight.grad * learning_rate
lr = 0.01
new_weight = weight.data - weight.grad * lr
new_bias = bias.data - bias.grad * lr

In [30]:
print(new_weight.data, weight.data)
print(new_bias.data, bias.data)

tensor([[1.3045]]) tensor([[-0.6871]])
tensor([-0.6625]) tensor([-1.5204])


In [21]:
pred2 = X_train @ new_weight + new_bias
loss2 = torch.mean((y_train - pred2)**2)

In [22]:
print("이전 loss:", loss)
print("update후 loss:", loss2)

이전 loss: tensor(2227.4529, grad_fn=<MeanBackward0>)
update후 loss: tensor(1762.5054)


### 모델링

In [51]:
weight = torch.randn(1, 1, requires_grad=True)
bias = torch.randn(1, requires_grad=True)

In [52]:
# 추론하는 함수 (예측 모델)
def linear_model(X):
    return X @ weight + bias

# 오차계산함수 (MSE)
def loss_fn(pred, y):
    return torch.mean((pred-y)**2)

### 학습
1. 모델을 이용해 추정한다.
   - pred = model(input)
1. loss를 계산한다.
   - loss = loss_fn(pred, target)
1. 계산된 loss를 파라미터에 대해 미분하여 계산한 gradient 값을 각 파라미터에 저장한다.
   - loss.backward()
1. optimizer를 이용해 파라미터를 update한다.
   - optimizer.step()  
1. 파라미터의 gradient(미분값)을 0으로 초기화한다.
   - optimizer.zero_grad()
- 위의 단계를 반복한다.   

In [54]:
epochs = 2000 # 반복횟수
lr = 0.01
# lr = 1000
# lr = 0.00000001
for epoch in range(epochs):
    # 1. 추정
    pred = linear_model(X_train)
    # 2. loss 계산
    loss = loss_fn(pred, y_train)
    # 3. parameter들의 gradient를 계산 (loss에대한 변화)
    loss.backward()
    # 4. parameter update
    weight.data = weight.data - weight.grad * lr # .data:현재값, .grad: gradient계산값.
    bias.data = bias.data - bias.grad * lr
    # 5. gradient 초기화
    weight.grad = None
    bias.grad = None
    ## 로그 - loss를 출력
    if epoch % 100 == 0 or epoch == (epochs-1):
        print(f"[{epoch+1}/{epochs}] loss: {loss.item():.5f}")

[1/2000] loss: 1982.44788
[101/2000] loss: 4.13011
[201/2000] loss: 2.55215
[301/2000] loss: 1.57707
[401/2000] loss: 0.97453
[501/2000] loss: 0.60220
[601/2000] loss: 0.37212
[701/2000] loss: 0.22995
[801/2000] loss: 0.14209
[901/2000] loss: 0.08781
[1001/2000] loss: 0.05426
[1101/2000] loss: 0.03353
[1201/2000] loss: 0.02072
[1301/2000] loss: 0.01280
[1401/2000] loss: 0.00791
[1501/2000] loss: 0.00489
[1601/2000] loss: 0.00302
[1701/2000] loss: 0.00187
[1801/2000] loss: 0.00115
[1901/2000] loss: 0.00071
[2000/2000] loss: 0.00044


In [42]:
weight.data, bias.data

(tensor([[19.9979]]), tensor([0.0047]))

In [47]:
new_X = torch.tensor([[5], [4], [2]], dtype=torch.float32)
linear_model(new_X)

tensor([[99.9944],
        [79.9964],
        [40.0006]], grad_fn=<AddBackward0>)

# 다중 입력, 다중 출력
- 다중입력: Feature가 여러개인 경우
- 다중출력: Output 결과가 여러개인 경우

다음 가상 데이터를 이용해 사과와 오렌지 수확량을 예측하는 선형회귀 모델을 정의한다.  
[참조](https://www.kaggle.com/code/aakashns/pytorch-basics-linear-regression-from-scratch)


|온도(F)|강수량(mm)|습도(%)|사과생산량(ton)|오렌지생산량|
|-|-|-|-:|-:|
|73|67|43|56|70|
|91|88|64|81|101|
|87|134|58|119|133|
|102|43|37|22|37|
|69|96|70|103|119|

```
사과수확량  = w11 * 온도 + w12 * 강수량 + w13 * 습도 + b1
오렌지수확량 = w21 * 온도 + w22 * 강수량 + w23 *습도 + b2
```

- `온도`, `강수량`, `습도` 값이 **사과**와, **오렌지 수확량**에 어느정도 영향을 주는지 가중치를 찾는다.
    - 모델은 사과의 수확량, 오렌지의 수확량 **두개의 예측결과를 출력**해야 한다.
    - 사과에 대해 예측하기 위한 weight 3개와 오렌지에 대해 예측하기 위한 weight 3개 이렇게 두 묶음, 총 6개의 weight를 정의하고 학습을 통해 가장 적당한 값을 찾는다.
        - `개별 과일를 예측하기 위한 weight들 @ feature들` 의 계산 결과를  **Node, Unit, Neuron** 이라고 한다.
        - 두 과일에 대한 Unit들을 묶어서 **Layer** 라고 한다.
- 목적은 우리가 수집한 train 데이터셋을 이용해 **정확한 예측을 위한 weight와 bias 들**을 찾는 것이다.

## Train Dataset
- Train data는 feature(input)와 target(output) 각각 2개의 행렬로 구성한다.
- Feature의 행은 관측치(개별 데이터)를 열을 Feature(특성, 변수)를 표현한다. 이 문제에서는 `온도, 강수량, 습도` 세개의 변수를 가진다.
- Target은 모델이 예측할 대상으로 행은 개별 관측치, 열은 각 항목에 대한 정답으로 구성한다. 이 문제에서 예측할 항목은 `사과수확량, 오렌지 수확량` 2개의 값이다.

In [55]:
#  input: 생산환경 (temp, rainfall, humidity) : (5, 3)
environs = [
    [73, 67, 43], 
    [91, 88, 64], 
    [87, 134, 58], 
    [102, 43, 37], 
    [69, 96, 70]
]

# Targets: 생산량 - (apples, oranges) - (5, 2)
apple_orange_output = [
    [56, 70], 
    [81, 101], 
    [119, 133], 
    [22, 37], 
    [103, 119]
]

In [56]:
# Dataset을 torch.Tensor로 생성
X = torch.tensor(environs, dtype=torch.float32)
y = torch.tensor(apple_orange_output, dtype=torch.float32)
X.shape, y.shape

(torch.Size([5, 3]), torch.Size([5, 2]))

In [57]:
X

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

## weight와 bias
- weight: 각 feature들이 생산량에 영향을 주었는지의 가중치로 feature에 곱해줄 값.
    - 사과, 오렌지의 생산량을 구해야 하므로 가중치가 두개가 된다.
    - weight의 shape: `(3, 2)`
- bias는 모든 feature들이 0일때 생산량이 얼마일지를 나타내는 값으로 feature와 weight간의 가중합 결과에 더해줄 값이다.
    - 사과, 오렌지의 생산량을 구하므로 bias가 두개가 된다.
    - bias의 shape: `(2, )`

### Linear Regression model
모델은 weights `w`와 inputs `x`의 내적(dot product)한 값에 bias `b`를 더하는 함수.

$$
\hspace{2.5cm} X \hspace{1.1cm} \cdot \hspace{1.2cm} W \hspace{1.2cm}  + \hspace{1cm} b \hspace{2cm}
$$

$$
\left[ \begin{array}{cc}
73 & 67 & 43 \\
91 & 88 & 64 \\
\vdots & \vdots & \vdots \\
69 & 96 & 70
\end{array} \right]
%
\cdot
%
\left[ \begin{array}{cc}
w_{11} & w_{21} \\
w_{12} & w_{22} \\
w_{13} & w_{23}
\end{array} \right]
%
+
%
\left[ \begin{array}{cc}
b_{1} & b_{2} \\
b_{1} & b_{2} \\
\vdots & \vdots \\
b_{1} & b_{2} \\
\end{array} \right]
$$


<center style="font-size:0.9em">
$w_{11},\,w_{12},\,w_{13}$: 사과 생산량 계산시 각 feature들(생산환경)에 곱할 가중치   <br>
$w_{21},\,w_{22},\,w_{23}$: 오렌지 생산량 계산시 각 feature들(생산환경)에 곱할 가중치    
</center>

<center>
<img src="figures/3_unit_layer.png">
</center>

In [87]:
# weight/bias 를 정의 -> 초기값은 random 값을 이용해서 생성.
weight = torch.randn(3, 2, requires_grad=True)
bias = torch.randn(2, requires_grad=True)

weight.size(), bias.size()
# weight: (3:input feature개수  ,  2:output 개수)
# bias: (2:output 개수, )

(torch.Size([3, 2]), torch.Size([2]))

In [88]:
weight

tensor([[-0.1575,  0.9041],
        [ 0.9811, -0.0167],
        [-0.6411, -0.6732]], requires_grad=True)

In [89]:
bias

tensor([0.5808, 0.8419], requires_grad=True)

In [90]:
X

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [91]:
# 한번 학습(최적화)
## 추론
pred = X @ weight + bias

In [92]:
pred

tensor([[27.2482, 36.7768],
        [31.5527, 38.5633],
        [81.1584, 38.2158],
        [ 2.9820, 67.4377],
        [39.0194, 14.4992]], grad_fn=<AddBackward0>)

In [93]:
y

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [94]:
## loss 계산(MSE)
loss = torch.mean((pred - y)**2) # 전체 추론한 결과의 평균오차를 계산.
loss

tensor(3499.1926, grad_fn=<MeanBackward0>)

In [95]:
# loss를 가지고 파라미터들(weight들, bias들)의 gradient 계산.
loss.backward()

In [97]:
weight.data

tensor([[-0.1575,  0.9041],
        [ 0.9811, -0.0167],
        [-0.6411, -0.6732]])

In [98]:
weight.grad

tensor([[-3249.0632, -4091.8333],
        [-3661.6863, -5828.9434],
        [-2355.6167, -3422.1777]])

In [101]:
bias.data, bias.grad

(tensor([0.5808, 0.8419]), tensor([-39.8079, -52.9014]))

In [102]:
# 파라미터 업데이트
lr = 0.00001
weight.data = weight.data - lr * weight.grad
bias.data = bias.data - lr * bias.grad

In [103]:
## 업데이트된 파라미터로 추정 -> loss 계산
pred2 = X @ weight + bias
loss2 = torch.mean((pred2 - y)**2)

In [105]:
print(loss.item(), loss2.item())

3499.192626953125 2661.15185546875


##  모델링

In [107]:
weight = torch.randn(3, 2, requires_grad=True)
bias = torch.randn(2, requires_grad=True)

## 모델 정의
def model(X):
    return X @ weight + bias

## loss 함수(MSE)
def loss_fn(pred, y):
    return torch.mean((pred - y)**2) # 전체 오차의 평균.

In [120]:
epochs = 5000
lr = 0.00001  # 1e-5
for epoch in range(epochs):
    # 1. 추론
    pred = model(X)
    # 2. loss 계산
    loss = loss_fn(pred, y)
    # 3. 파라미터 들의 gradient 계산
    loss.backward()
    # 4. 파라미터 업데이트
    weight.data = weight.data - lr * weight.grad
    bias.data = bias.data - lr * bias.grad
    # 5. gradient 초기화
    weight.grad = None
    bias.grad = None
    ## 100 epoch, 마지막 epoch에서 loss를 출력 => 학습 과정 log를 출력
    if epoch % 100 == 0 or epoch == epochs-1:
        print(f"[{epoch+1:04d}/{epochs}] - {loss.item():.5f}")

[0001/5000] - 0.54958
[0101/5000] - 0.54940
[0201/5000] - 0.54924
[0301/5000] - 0.54911
[0401/5000] - 0.54901
[0501/5000] - 0.54892
[0601/5000] - 0.54884
[0701/5000] - 0.54878
[0801/5000] - 0.54873
[0901/5000] - 0.54869
[1001/5000] - 0.54865
[1101/5000] - 0.54862
[1201/5000] - 0.54860
[1301/5000] - 0.54858
[1401/5000] - 0.54856
[1501/5000] - 0.54854
[1601/5000] - 0.54853
[1701/5000] - 0.54852
[1801/5000] - 0.54851
[1901/5000] - 0.54850
[2001/5000] - 0.54850
[2101/5000] - 0.54849
[2201/5000] - 0.54849
[2301/5000] - 0.54848
[2401/5000] - 0.54848
[2501/5000] - 0.54848
[2601/5000] - 0.54848
[2701/5000] - 0.54847
[2801/5000] - 0.54847
[2901/5000] - 0.54847
[3001/5000] - 0.54847
[3101/5000] - 0.54846
[3201/5000] - 0.54846
[3301/5000] - 0.54846
[3401/5000] - 0.54846
[3501/5000] - 0.54846
[3601/5000] - 0.54846
[3701/5000] - 0.54846
[3801/5000] - 0.54845
[3901/5000] - 0.54845
[4001/5000] - 0.54845
[4101/5000] - 0.54845
[4201/5000] - 0.54845
[4301/5000] - 0.54845
[4401/5000] - 0.54845
[4501/5000

In [113]:
# 새로운 데이터로 추론
p = model(X)
p

tensor([[ 57.4287,  70.3354],
        [ 82.0229, 100.6778],
        [118.6928, 132.8992],
        [ 21.0604,  36.9963],
        [101.9201, 119.1969]], grad_fn=<AddBackward0>)

In [114]:
y

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [118]:
new_x = torch.tensor([68, 82, 56], dtype=torch.float32)
new_x = new_x.unsqueeze(dim=0)
new_x.shape

torch.Size([1, 3])

In [119]:
model(new_x)

tensor([[80.9965, 95.6196]], grad_fn=<AddBackward0>)

# pytorch built-in 모델을 사용해 Linear Regression 구현

In [121]:
inputs = torch.tensor(
    [[73, 67, 43], 
     [91, 88, 64], 
     [87, 134, 58], 
     [102, 43, 37], 
     [69, 96, 70]], dtype=torch.float32)

In [122]:
targets = torch.tensor(
    [[56, 70], 
    [81, 101], 
    [119, 133], 
    [22, 37], 
    [103, 119]], dtype=torch.float32)

## nn.Linear
Pytorch는 nn.Linear 클래스를 통해 Linear Regression 모델을 제공한다.  
nn.Linear에 입력 feature의 개수와 출력 값의 개수를 지정하면 random 값으로 초기화한 weight와 bias들을 생성해 모델을 구성한다.

In [ ]:
# import torch

# torch.nn.Linear(input feature개수 , output 개수)   # weight, bias, X@weight  + bias

## Optimizer와 Loss 함수 정의
- torch.optim 모듈에 다양한 Optimizer 클래스가 구현되있다.
- torch.nn 또는 torch.nn.functional 모듈에 다양한 Loss 함수가 제공된다. 이중 mse_loss() 를 사용한다.

In [125]:
# 선형회귀 모델을 정의. torch.nn.Linear 클래스
import torch
import torch.nn as nn

model = nn.Linear(3, 2)  # 3: input feature 개수, 2: output 수수
# def model(X):

In [126]:
# loss 함수
loss_fn = torch.nn.functional.mse_loss # 함수 
# loss_fn = torch.nn.MSELoss()  # 클래스 -> 객체를 생성
# def loss_fn(pred, y):

In [130]:
# optimizer (torch.optim 모듈에 정의):    weight.data = weight.data - lr * weight.grad
optimizer = torch.optim.SGD(
    model.parameters(), # 최적화 대상 파라미터들을 model에서 조회해서 전달.
    lr = 0.00001,  # Learning Rage
)

In [129]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.4338,  0.5429, -0.2817],
         [ 0.2112, -0.5735,  0.1724]], requires_grad=True),
 Parameter containing:
 tensor([-0.1661, -0.3176], requires_grad=True)]

## Model Train

In [132]:
epochs = 5000

for epoch in range(epochs):
    # 추론
    pred = model(inputs)
    # loss 계산
    loss = loss_fn(pred, targets) # torch.nn.functional.mse_loss(pred, targets) # (모델추정값, 정답)
    # gradient 계산
    loss.backward()
    # 파라미터 업데이트: optimizer.step()
    optimizer.step()
    # 파라미터 초기화 w.grad=None, b.grad=None
    optimizer.zero_grad()
    # 현재 epoch 학습 결과를 log로 출력
    if epoch % 100 == 0 or epoch == epochs-1:
        print(f"[{epoch+1:04d}/{epochs}] - {loss.item()}")

[0001/5000] - 0.5096172094345093
[0101/5000] - 0.5091460943222046
[0201/5000] - 0.5087567567825317
[0301/5000] - 0.5084331631660461
[0401/5000] - 0.508165180683136
[0501/5000] - 0.5079387426376343
[0601/5000] - 0.5077613592147827
[0701/5000] - 0.5076063275337219
[0801/5000] - 0.5074799060821533
[0901/5000] - 0.507374107837677
[1001/5000] - 0.5072866082191467
[1101/5000] - 0.5072098970413208
[1201/5000] - 0.5071526765823364
[1301/5000] - 0.5071009397506714
[1401/5000] - 0.5070592164993286
[1501/5000] - 0.5070251822471619
[1601/5000] - 0.5069953203201294
[1701/5000] - 0.5069748163223267
[1801/5000] - 0.5069497227668762
[1901/5000] - 0.5069342255592346
[2001/5000] - 0.5069197416305542
[2101/5000] - 0.5069082975387573
[2201/5000] - 0.506901204586029
[2301/5000] - 0.5068908929824829
[2401/5000] - 0.5068827271461487
[2501/5000] - 0.5068793892860413
[2601/5000] - 0.5068731904029846
[2701/5000] - 0.5068670511245728
[2801/5000] - 0.5068671107292175
[2901/5000] - 0.5068622827529907
[3001/5000] -

In [137]:
# 추론 => gradient 계산을 할 필요가 없다. ==> grad_fn을 만들 필요가 없다.
with torch.no_grad():
    pred = model(inputs)

In [138]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [139]:
pred

tensor([[ 57.1382,  70.2557],
        [ 82.2236, 100.7036],
        [118.6980, 132.9673],
        [ 21.0851,  37.0220],
        [101.9156, 119.1292]])

In [140]:
# 학습 로직을 함수 구현
def train(inputs, targets, epochs, model, loss_fn, optimizer):

    for epoch in range(epochs):
        # 추론
        pred = model(inputs)
        # loss 계산
        loss = loss_fn(pred, targets) # torch.nn.functional.mse_loss(pred, targets) # (모델추정값, 정답)
        # gradient 계산
        loss.backward()
        # 파라미터 업데이트: optimizer.step()
        optimizer.step()
        # 파라미터 초기화 w.grad=None, b.grad=None
        optimizer.zero_grad()
        # 현재 epoch 학습 결과를 log로 출력
        if epoch % 100 == 0 or epoch == epochs-1:
            print(f"[{epoch+1:04d}/{epochs}] - {loss.item()}")

In [141]:
model = nn.Linear(3, 2)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001)

In [142]:
train(inputs, targets, 5000, model, nn.functional.mse_loss, optimizer)

[0001/5000] - 18969.671875
[0101/5000] - 10.212318420410156
[0201/5000] - 2.0050790309906006
[0301/5000] - 0.7476084232330322
[0401/5000] - 0.5549023747444153
[0501/5000] - 0.525364339351654
[0601/5000] - 0.5208339095115662
[0701/5000] - 0.5201326608657837
[0801/5000] - 0.5200199484825134
[0901/5000] - 0.5199987888336182
[1001/5000] - 0.5199880003929138
[1101/5000] - 0.5199852585792542
[1201/5000] - 0.5199781060218811
[1301/5000] - 0.5199733972549438
[1401/5000] - 0.5199659466743469
[1501/5000] - 0.519959568977356
[1601/5000] - 0.5199530124664307
[1701/5000] - 0.5199490785598755
[1801/5000] - 0.5199415683746338
[1901/5000] - 0.519936203956604
[2001/5000] - 0.5199282169342041
[2101/5000] - 0.5199248194694519
[2201/5000] - 0.5199151039123535
[2301/5000] - 0.5199094414710999
[2401/5000] - 0.5199055671691895
[2501/5000] - 0.5198995471000671
[2601/5000] - 0.5198938250541687
[2701/5000] - 0.5198862552642822
[2801/5000] - 0.5198787450790405
[2901/5000] - 0.5198774933815002
[3001/5000] - 0.519